In [1]:
# import os
# os.environ['TOKENIZERS_PARALLELISM']='false'

In [2]:
from datasets import load_dataset
from transformers import LayoutLMv2ImageProcessor, LayoutXLMTokenizerFast
from transformers.models.lilt.modeling_lilt import LiltForPretraining
import torch
from data_collator import DataCollatorForLiltPretraining
from tqdm import tqdm

In [3]:
rvl_cdip = load_dataset('rvl_cdip',split='train')

Found cached dataset rvl_cdip (/home/hj36wegi/scratch/data/huggingface/datasets/rvl_cdip/default/1.0.0/ea410993ed3f5b9744d8616ffbaad5f70a75a21a4233626dd07b3de31d381e53)


In [4]:
img_processor = LayoutLMv2ImageProcessor(do_resize=False,
                                         do_rescale=False,
                                         do_normalize=False,
                                         ocr_lang='eng',
                                         )
tokenizer = LayoutXLMTokenizerFast.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base")

In [5]:
def process_example(example):
    
    images = [img.convert('RGB') for img in example['image']]
    
    features = img_processor(images)
    encoding = tokenizer(
        text=features["words"],
        boxes=features["boxes"],
        return_special_tokens_mask=True,
        return_tensors='pt',
        max_length=512,
        padding='max_length',
        truncation=True
    )
    # encoding.pop('image')
    # encoding['image']=example['image']
    # encoding['category']=torch.LongTensor(example['label'])
    # encoding['words']=features["words"]
    return encoding

In [6]:
rvl_cdip.set_transform(process_example)

In [7]:
rvl_cdip[0].keys()

dict_keys(['input_ids', 'attention_mask', 'special_tokens_mask', 'bbox'])

In [8]:
for k,v in rvl_cdip[0].items():
    print(v.shape)

torch.Size([512])
torch.Size([512])
torch.Size([512])
torch.Size([512, 4])


In [9]:
collator = DataCollatorForLiltPretraining(tokenizer=tokenizer,seq_len=512)

In [10]:
loader = torch.utils.data.DataLoader(rvl_cdip,collate_fn=collator,batch_size=8,num_workers=8,shuffle=False)

In [11]:
batch = next(iter(loader))

for k,v in batch.items():
    print(v.shape)

In [12]:
for batch in tqdm(loader,total=len(loader)):
    if batch['attention_mask'].max()>1:
        break

  4%|██████▋                                                                                                                                                       | 1682/40000 [19:21<7:21:03,  1.45it/s]


KeyboardInterrupt: 

In [ ]:
model = LiltForPretraining.from_pretrained("SCUT-DLVCLab/lilt-infoxlm-base").cuda()

In [ ]:
for idx,batch in tqdm(enumerate(loader),total=len(loader)):
    # batch.pop('category')
    batch = {k:v.cuda() for k,v in batch.items()}
    with torch.no_grad():
        out = model(**batch)

In [ ]:
batch = next(iter(loader))